In [1]:
import pandas as pd

In [2]:
# Cesty k súborom
file_control    = r"C:\Users\marti\Diplomovka\DiplomovkaOpravene.xlsx"
file_stanza     = r"C:\Users\marti\Diplomovka\stanza\outputStanza.xlsx"
file_trankit    = r"C:\Users\marti\Diplomovka\Trankit\Trankit_new_words.xlsx"
file_treetagger = r"C:\Users\marti\Diplomovka\TreeTagger\outputTreeTagger.xlsx"
file_udpipe     = r"C:\Users\marti\Diplomovka\UDPipe\UDPipe.xlsx"

In [3]:
# Načítame kontrolný súbor
control_df = pd.read_excel(file_control)
n = len(control_df)  # počet riadkov, ktoré budeme používať (predpokladáme, že ostatné súbory budú zreťazené podľa kontrolného)

# Vytvoríme základný výstupný DataFrame – najprv pridáme Token zo súboru kontrol
output_df = pd.DataFrame()
output_df['Token'] = control_df['Token'].iloc[:n].reset_index(drop=True)

In [4]:
# Definujeme informácie o knižniciach: kľúč = názov knižnice, hodnota = (cesta, prefix pre stĺpce)
libraries = {
    'stanza': (file_stanza, "Xpos"),
    'trankit': (file_trankit, "Xpos"),  # pre trankit používame veľké "XPOS"
    'treetagger': (file_treetagger, "Xpos"),
    'udpipe': (file_udpipe, "Xpos")
}

# Pre každú knižnicu načítame jej súbor, orežeme na n riadkov a pre každý Xpos (1 až 7) porovnáme hodnotu s kontrolným
for lib, (file_path, prefix) in libraries.items():
    # Načítame súbor danej knižnice
    lib_df = pd.read_excel(file_path)
    lib_df = lib_df.iloc[:n].reset_index(drop=True)
    
    # Pre každý Xpos index od 1 do 7
    for i in range(1, 8):
        # Vytvoríme názov stĺpca pre danú knižnicu (napr. "stanza1" alebo "trankit1")
        out_col = f"{lib}{i}"
        # Predpokladáme, že v kontrolnom súbore sú stĺpce "Xpos1", "Xpos2", …, "Xpos7"
        control_col = f"Xpos{i}"
        # Pre knižnice s prefixom "XPOS" (trankit) použijeme tento názov; pre ostatné predpokladáme "Xpos"
        lib_col = f"{prefix}{i}"
        
        # Porovnanie – previesť hodnoty na reťazce, odstrániť medzery a prepnúť na malé písmená
        control_vals = control_df[control_col].astype(str).str.strip().str.lower().reset_index(drop=True)
        lib_vals = lib_df[lib_col].astype(str).str.strip().str.lower().reset_index(drop=True)
        
        # Vytvoríme booleovský stĺpec, ktorý bude True, ak sa hodnoty zhodujú, inak False
        output_df[out_col] = (lib_vals == control_vals)

# Uložíme výsledný DataFrame do Excelu
output_path = r"C:\Users\marti\Diplomovka\Agregovany_XPOS_Match.xlsx"
output_df.to_excel(output_path, index=False)

print("Výsledný DataFrame uložený do:", output_path)


Výsledný DataFrame uložený do: C:\Users\marti\Diplomovka\Agregovany_XPOS_Match.xlsx


In [6]:
# Predpokladáme, že output_df obsahuje stĺpce:
# "Token", "stanza1", "stanza2", ... "stanza7",
# "trankit1", ... "trankit7", "treetagger1", ... "treetagger7",
# "udpipe1", ... "udpipe7", "morphodita1", ... "morphodita7"
# a v týchto stĺpcoch sú booleovské hodnoty True/False.

libraries = ['stanza', 'trankit', 'treetagger', 'udpipe']
percent_summary = {}

for lib in libraries:
    lib_summary = {}
    for i in range(1, 8):
        col = f"{lib}{i}"
        if col in output_df.columns:
            # .mean() pre booleovské stĺpce vráti podiel True (keďže True = 1, False = 0)
            percentage = output_df[col].mean() * 100
            lib_summary[f"Xpos{i}"] = percentage
    percent_summary[lib] = lib_summary

print("Percentuálne vyhodnotenie zhody pre jednotlivé knižnice a pozície Xpos:")
for lib, summary in percent_summary.items():
    print(f"{lib}:")
    for xpos, pct in summary.items():
        print(f"  {xpos}: {pct:.2f}%")


Percentuálne vyhodnotenie zhody pre jednotlivé knižnice a pozície Xpos:
stanza:
  Xpos1: 97.00%
  Xpos2: 96.16%
  Xpos3: 91.58%
  Xpos4: 97.25%
  Xpos5: 94.33%
  Xpos6: 90.58%
  Xpos7: 97.75%
trankit:
  Xpos1: 98.25%
  Xpos2: 96.58%
  Xpos3: 91.66%
  Xpos4: 97.16%
  Xpos5: 95.50%
  Xpos6: 91.08%
  Xpos7: 97.91%
treetagger:
  Xpos1: 29.11%
  Xpos2: 28.27%
  Xpos3: 28.86%
  Xpos4: 44.04%
  Xpos5: 33.94%
  Xpos6: 62.89%
  Xpos7: 88.66%
udpipe:
  Xpos1: 97.91%
  Xpos2: 96.83%
  Xpos3: 91.24%
  Xpos4: 97.25%
  Xpos5: 95.00%
  Xpos6: 90.99%
  Xpos7: 98.00%


In [8]:
# Predpokladáme, že aggregatedUPOS je DataFrame s požadovanými stĺpcami
libraries = ['stanza', 'trankit', 'treetagger', 'udpipe']
percent_summary = {}

for lib in libraries:
    lib_summary = {}
    
    # Pre Xpos1: počítame normálne
    col1 = f"{lib}1"
    percentage = output_df[col1].mean() * 100  # .mean() vracia podiel True
    lib_summary["Xpos1"] = percentage
    
    # Pre Xpos2 až Xpos7: počítame len pre tie tokeny, kde je zhoda v prvej pozícii
    for i in range(2, 8):
        col = f"{lib}{i}"
        # Vyberieme tie riadky, kde je zhoda pre pozíciu 1
        subset = output_df[output_df[col1] == True]
        if len(subset) > 0:
            percentage = subset[col].mean() * 100
        else:
            percentage = 0
        lib_summary[f"Xpos{i}"] = percentage
        
    percent_summary[lib] = lib_summary

# Výpis výsledkov
print("Percentuálne vyhodnotenie zhody pre jednotlivé knižnice a pozície Xpos (porovnávané len, ak Xpos1 je správne):")
for lib, summary in percent_summary.items():
    print(f"{lib}:")
    for xpos, pct in summary.items():
        print(f"  {xpos}: {pct:.2f}%")


Percentuálne vyhodnotenie zhody pre jednotlivé knižnice a pozície Xpos (porovnávané len, ak Xpos1 je správne):
stanza:
  Xpos1: 97.00%
  Xpos2: 98.19%
  Xpos3: 92.26%
  Xpos4: 97.94%
  Xpos5: 95.10%
  Xpos6: 91.49%
  Xpos7: 98.11%
trankit:
  Xpos1: 98.25%
  Xpos2: 97.71%
  Xpos3: 92.28%
  Xpos4: 97.62%
  Xpos5: 95.93%
  Xpos6: 91.85%
  Xpos7: 98.05%
treetagger:
  Xpos1: 29.11%
  Xpos2: 82.81%
  Xpos3: 66.76%
  Xpos4: 78.51%
  Xpos5: 65.62%
  Xpos6: 85.96%
  Xpos7: 93.98%
udpipe:
  Xpos1: 97.91%
  Xpos2: 97.96%
  Xpos3: 91.91%
  Xpos4: 97.79%
  Xpos5: 95.49%
  Xpos6: 91.82%
  Xpos7: 98.13%
